In [1]:
import os
from glob import glob
import pandas as pd
import numpy as np

In [2]:
# path
path = os.path.abspath(r'C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\3_Data')
d_path_dic = {'1_Behavior':'0_raw','2_Biosignal':'0_source', '3_MRI':'0_sourcedata'}
t_path = os.path.abspath(r'C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\8._Article\1_fMRI_1st\data')

In [3]:
# sub_dictionary
# CRF="C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\3_Data\4_CRF\(혁신형, 창의도전)_CRF(20230811).xlsx"
# key: subject number
# value: list [group, number of session]

sub_dic = {'sub-0001':[0,2],'sub-0002':[0,2],'sub-0003':[0,2],'sub-0004':[0,2],'sub-0005':[0,1],'sub-0006':[0,2],
       'sub-0008':[0,2],'sub-0011':[0,2],'sub-0013':[0,2],'sub-0014':[0,2],'sub-0020':[0,2],'sub-0022':[0,2],
       'sub-0024':[0,2],'sub-0025':[0,2],'sub-0026':[0,2],'sub-0029':[0,2],'sub-0030':[0,1],'sub-0031':[0,2],
       'sub-0034':[0,2],'sub-0038':[0,2],'sub-0044':[0,2],'sub-0046':[0,2],'sub-0048':[0,2],'sub-0050':[0,2],
       'sub-0051':[0,2],'sub-0053':[0,2],'sub-0054':[0,2],'sub-0055':[0,2],'sub-0058':[0,2],'sub-0059':[0,1],
       'sub-0007':[1,4],'sub-0009':[1,4],'sub-0012':[1,4],'sub-0016':[1,4],'sub-0017':[1,4],'sub-0018':[1,4],
       'sub-0021':[1,4],'sub-0023':[1,4],'sub-0028':[1,4],'sub-0032':[1,4],'sub-0033':[1,4],'sub-0035':[1,4],
       'sub-0036':[1,4],'sub-0037':[1,4],'sub-0039':[1,4],'sub-0040':[1,4],'sub-0041':[1,4],'sub-0042':[1,4],
       'sub-0043':[1,4],'sub-0045':[1,4],'sub-0047':[1,4],'sub-0049':[1,4],'sub-0057':[1,4],'sub-0062':[1,4],
       'sub-0063':[1,4]}



In [4]:

class BiosingnalExt:
    def set_paths (self, data_path,sub_dic): 
        """
        generates a list of paths that exist in the data_path
        Args:
        - data_path (string): Input path of raw biosignal folder.                
        Returns:
        - Biosignal_paths = dictionary: key = sub-00xx_ses-0x, value = path 

        # reference: ref1 : 폴더명 생성 규칙 # ref2 : Information and location by file
        """
        Biosignal_paths = {}
        subs = os.listdir(data_path) 
        for sub in subs:
            if sub in sub_dic.keys():                    
                path2 = os.path.join(data_path,sub)
                sess = os.listdir(path2) 
                for ses in sess:
                    key = '_'.join([sub,ses])
                    path3 = os.path.join(path2,ses)
                    dirs = os.listdir(path3)
                    f_list = []
                    for dir in dirs:
                        path_dir = os.path.join(path3, dir)
                        f_list.extend(glob(os.path.join(path_dir, '*.csv')))
                        f_list.extend(glob(os.path.join(path_dir, '*.txt')))
                    Biosignal_paths['_'.join([sub,ses])] = f_list
            else: pass
        
        self.Biosignal_paths = Biosignal_paths

    def hrv_r_slicer (self,x):
        """
        generates value and naume related to HRV from 'HRV_results.txt' 
        Args:
        - x (string): string from HRV_Result                
        Returns 
        - (string) name & val of x from HRV_Results

        # reference: ref3 : Information and location by file
        """
        eq_ind = x.find('=') 
        name = x[:eq_ind]
        after_eq = x[eq_ind+1:] 
        if '(' in after_eq:
            br_ind = after_eq.find('(')
            val = x[eq_ind+2:eq_ind+br_ind+1] 
        else:
            val = x[eq_ind+2:]
        return name, val
    

    def gen_vrabes_dict (self):
        """
        generates a dataframe with dictionary, index = sub-00xx_ses-0x / columns, HRV and anx score

        Args:
        - self (self.Biosignal_paths): paths of files              
        Returns 
        - self.biosignal_df :
        """
        f_path = self.Biosignal_paths
        all_dic = {}
        for key in f_path.keys():      
            subses_dic = {}       
            for file_path in f_path[key]:            
                filename = file_path.split('\\')[-1] # filename
                if filename[-3:] == 'csv':
                    csv_df = pd.read_csv(file_path,header=1)
                    col1 = filename[:-4]
                    if filename[-7:-4] =='HRV':
                        col2 = csv_df.columns[0]
                        start_time = csv_df.columns[1]
                        subses_dic['_'.join([col1,col2])] = start_time
                    else:
                        for d in csv_df.columns:
                            score = csv_df[d][0]
                            subses_dic['_'.join([col1,d])] = score  
                elif filename[-3:] == 'txt':
                    hrv_result = pd.read_csv(file_path,header=None)
                    col1 = filename[:-4]
                    for e in hrv_result.loc[:,0]:
                        col2, hrv = self.hrv_r_slicer(e)
                        subses_dic['_'.join([col1,col2])] = hrv
                all_dic[key] = subses_dic
            biosignal_df = pd.DataFrame(all_dic)
        self.biosignal_df = biosignal_df.transpose()

    def missing_ (self):
        """
        generates a dataframe with dictionary, index = sub-00xx_ses-0x / columns, HRV and anx score
        Args:
        - self (self.Biosignal_paths): paths of files              
        Returns 
        - self.biosignal_dic :
        """
        df = pd.DataFrame()
        df['nan'] = self.biosignal_df.apply(lambda x: x.isnull().sum(), axis=1)
        df['nan_list'] = self.biosignal_df.apply(lambda x: list(x.index[x.isnull()]), axis=1)
        # Find the index of the row where the column 'nan' is non-zero
        non_zero_nan_indices = df[df['nan'] != 0].index
        # Save the value of the 'nan_list' column in that index as a dictionary
        nan_list_dict = {index: df.loc[index, 'nan_list'] for index in non_zero_nan_indices}

        self.missing = nan_list_dict

In [5]:
test = os.path.join(path,'1_Behavior','0_raw')
a = BiosingnalExt()
a.set_paths(test,sub_dic)
a.gen_vrabes_dict()
biosignal_df = a.biosignal_df
a.missing_()
biosignal_info = {}
for key in a.Biosignal_paths.keys():
    biosignal_info[key] = len(a.Biosignal_paths[key])


In [10]:
biosignal_df.to_csv(os.path.join(path,'hrv_anxscr_230816.csv',))